# Ax optimize

In [27]:
from ax import optimize
best_parameters, best_values, experiment, model = optimize(
        parameters=[
          {
            "name": "x1",
            "type": "range",
            "bounds": [-10.0, 10.0],
          },
          {
            "name": "x2",
            "type": "range",
            "bounds": [-10.0, 10.0],
          },
        ],
        # Booth function
        evaluation_function=lambda p: (p["x1"] + 2*p["x2"] - 7)**2 + (2*p["x1"] + p["x2"] - 5)**2,
        minimize=True,
    )


[INFO 06-15 07:14:04] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x1. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 06-15 07:14:04] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x2. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 06-15 07:14:04] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='x1', parameter_type=FLOAT, range=[-10.0, 10.0]), RangeParameter(name='x2', parameter_type=FLOAT, range=[-10.0, 10.0])], parameter_constraints=[]).
[INFO 06-15 07:14:04] ax.modelbridge.dispatch_utils: Using Bayesian optimization since there are more ordered parameters than there are categories for the unordered categorical parameters.
[INFO 06-15 07:14:04] ax.modelbridge.dispatch_uti

In [3]:
best_parameters

{'x1': 1.0499033778313702, 'x2': 3.0411949204629796}

In [24]:
best_parameters, best_values, experiment, model = optimize(
        parameters=[
          {
            "name": "x1",
            "type": "range",
            "bounds": [-10.0, 10.0],
          },
          {
            "name": "x2",
            "type": "range",
            "bounds": [-10.0, 10.0],
          },
        ],
        # Booth function
        evaluation_function=lambda p: (p["x1"])**2 + (p["x2"] - 5)**2,
        minimize=False,
    )

[INFO 06-15 07:12:56] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x1. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 06-15 07:12:56] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x2. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 06-15 07:12:56] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='x1', parameter_type=FLOAT, range=[-10.0, 10.0]), RangeParameter(name='x2', parameter_type=FLOAT, range=[-10.0, 10.0])], parameter_constraints=[]).
[INFO 06-15 07:12:56] ax.modelbridge.dispatch_utils: Using Bayesian optimization since there are more ordered parameters than there are categories for the unordered categorical parameters.
[INFO 06-15 07:12:56] ax.modelbridge.dispatch_uti

In [26]:
best_parameters, best_values, experiment, model

({'x1': 10.0, 'x2': -10.0},
 ({'objective': 324.9381930848465},
  {'objective': {'objective': 0.8859988544696163}}),
 Experiment(None),
 <ax.modelbridge.torch.TorchModelBridge at 0x7f663ff22cd0>)

In [23]:
from typing import Any, Dict, Optional, Tuple, Type

# Ax wrappers for BoTorch components
from ax.models.torch.botorch_modular.model import BoTorchModel
from ax.models.torch.botorch_modular.surrogate import Surrogate
from ax.models.torch.botorch_modular.list_surrogate import ListSurrogate
from ax.models.torch.botorch_modular.acquisition import Acquisition

# Ax data tranformation layer
from ax.modelbridge.torch import TorchModelBridge
from ax.modelbridge.registry import Cont_X_trans, Y_trans, Models

# Experiment examination utilities
from ax.service.utils.report_utils import exp_to_df

# Test Ax objects
from ax.utils.testing.core_stubs import (
    get_branin_experiment, 
    get_branin_data, 
    get_branin_experiment_with_multi_objective,
    get_branin_data_multi_objective,
)

# BoTorch components
from botorch.models.model import Model
from botorch.models.gp_regression import FixedNoiseGP, SingleTaskGP
from botorch.acquisition.monte_carlo import qExpectedImprovement, qNoisyExpectedImprovement
from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood

In [3]:
experiment = get_branin_experiment(with_trial=True)
data = get_branin_data(trials=[experiment.trials[0]])

[INFO 06-15 07:00:31] ax.core.experiment: The is_test flag has been set to True. This flag is meant purely for development and integration testing purposes. If you are running a live experiment, please set this flag to False


In [4]:
model_bridge_with_GPEI = Models.BOTORCH_MODULAR(
    experiment=experiment,
    data=data,
    surrogate=Surrogate(SingleTaskGP),  # Optional, will use default if unspecified
    botorch_acqf_class=qNoisyExpectedImprovement,  # Optional, will use default if unspecified
)

[INFO 06-15 07:00:32] ax.modelbridge.transforms.standardize_y: Outcome branin is constant, within tolerance.


In [5]:
generator_run = model_bridge_with_GPEI.gen(n=1)
generator_run.arms[0]

Arm(parameters={'x1': 10.0, 'x2': 0.0})

# scipy optimize

In [29]:
from scipy.optimize import minimize
fun = lambda x: (x[0] - 1)**2 + (x[1] - 2.5)**2
res = minimize(fun, (2, 0), method='powell')
res

   direc: array([[1., 0.],
       [0., 1.]])
     fun: 4.930380657631324e-32
 message: 'Optimization terminated successfully.'
    nfev: 36
     nit: 2
  status: 0
 success: True
       x: array([1. , 2.5])

# scipy optimize

In [ ]:
import numpy as np
from scipy import optimize

def func(x):
    return x**2

# bounds = optimize.Bounds(lb=-1000., ub=1000.)
# res = optimize.minimize(fun=func, x0=np.array([1]), method='Powell', bounds=((-1000., 1000.)))
# print(res)


fun = lambda x: (x[0] - 1)**2 + (x[1] - 2.5)**2
# res = optimize.minimize(fun, (2, 0), method='TNC', tol=1e-10)
# print(res)
bnds = ((0.25, 100), (0, 2.0))
res = optimize.minimize(fun=fun, x0=(2, 0), method='Powell', bounds=bnds)
print(res.x)

fun = lambda x: (x[0] - 1)**2
bnds = ((0.25, 100),)
res = optimize.minimize(fun=fun, x0=(2,), method='Powell', bounds=bnds)
print(res.x)

# sklearn random forest

In [12]:
# random forest
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import VotingRegressor
X, y = load_diabetes(return_X_y=True)

# Train classifiers
reg1 = GradientBoostingRegressor(random_state=1)
reg2 = RandomForestRegressor(n_estimators= 10, random_state=1)
reg3 = LinearRegression()

reg1.fit(X, y)
reg2.fit(X, y)
reg3.fit(X, y)

ereg = VotingRegressor([("gb", reg1), ("rf", reg2), ("lr", reg3)])
ereg.fit(X, y)

VotingRegressor(estimators=[('gb', GradientBoostingRegressor(random_state=1)),
                            ('rf',
                             RandomForestRegressor(n_estimators=10,
                                                   random_state=1)),
                            ('lr', LinearRegression())])

In [13]:
X.shape, y.shape

((442, 10), (442,))

In [15]:
xt = X[:20]
pred2 = reg2.predict(xt)
pred2

array([158.4,  80.1, 163.3, 197.9, 117.7, 105.1, 105.8,  87.1, 117.9,
       238.9, 106.8,  94.2, 149.5, 177.7, 113.7, 178.4, 207.7, 153. ,
        92.6, 145.5])

In [16]:
y[:20]

array([151.,  75., 141., 206., 135.,  97., 138.,  63., 110., 310., 101.,
        69., 179., 185., 118., 171., 166., 144.,  97., 168.])